# Midterm — iNaturalist
### Daniel Phillips, Chris Howard, Phillip Johnson, Jacob Smith, Michael Reid

import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
from IPython.display import display

def get_taxa_id(species_name):
    
    base_url = "http://api.inaturalist.org/v1/taxa/autocomplete?q="

    
    request = requests.get(base_url + "%20".join(species_name.split()))
    data = request.json()

    ids = []
    for i in data['results']:
        ids.append(i['id'])

    return ids

def get_observation(id_no, month, year):
    #Url builder, for the request
    base_url = "http://api.inaturalist.org/v1/observations?"
    end_url = "&order=desc&order_by=created_at"
    url = base_url + 'taxon_id=' + str(id_no) + '&month=' + str(month) + '&year=' + str(year) + end_url

    request = requests.get(url)
    data = request.json()    

    
    return data
    
        
def get_count_one_month(id_no_lst, month, year):
    
    count = 0
    for i in id_no_lst:
        count += int(get_observation(i, month, year)['total_results'])
    return count
    
    

species = [
    'Danaus plexippus',
    'Hyles lineata',
    'Zerene cesonia',
    'Papilio multicaudata',
    'Agraulis vanillae',
    'Papilio cresphontes',
    'Strymon melinus',
    'Vanessa cardui',
    'Hylephila phyleus',
    'Danaus gilippus'
]

months = [
    'January',
    'February',
    'March',
    'April',
    'May',
    'June',
    'July',
    'August',
    'September',
    'October',
    'November',
    'December'
]

def main():
    print('running')
    species_to_id = {}
    frames = []
    
    
    #Get a dictionary of the taxa -> lst(ids)
    for i in species:
        species_to_id[i] = get_taxa_id(i)
    
    month_map = dict(zip(range(1,13), months))
    
    species_dict_out = {}
    year = 2016
    
    print(species_to_id)
    
    #iterate through the 
    for spec in species_to_id:
        species_dict_out[spec] = {}
        
        for mon in month_map:
            species_dict_out[spec][month_map[mon]] = get_count_one_month(species_to_id[spec], mon, year)
        
        print(species_dict_out[spec])
        frames.append(species_dict_out[spec])
            
        
   
    result = json_normalize(frames)
    display(result)
    
    
main()
    